In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
%matplotlib inline


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### 1 Flow in 2D

In [5]:
def sample_data():
    count = 100000
    rand = np.random.RandomState(0)
    a = [[-1.5, 2.5]] + rand.randn(count // 3, 2) * 0.2
    b = [[1.5, 2.5]] + rand.randn(count // 3, 2) * 0.2
    c = np.c_[2 * np.cos(np.linspace(0, np.pi, count // 3)),
    -np.sin(np.linspace(0, np.pi, count // 3))]

    c += rand.randn(*c.shape) * 0.2
    data_x = np.concatenate([a, b, c], axis=0)
    data_y = np.array([0] * len(a) + [1] * len(b) + [2] * len(c))
    perm = rand.permutation(len(data_x))
    return data_x[perm], data_y[perm]

In [39]:
data_x, data_y = sample_data()
data_trn, data_val = data_x[:80000], data_x[80000:]
data_trn_y, data_val_y = data_y[:80000], data_y[80000:]

### 1.1

In [14]:
def build_mlp(layer_in, scope, num_layers, hidden_dim, output_dim, activation=tf.tanh, output_activation=None):
    '''
    layer_out = (None, k)
    '''
    with tf.variable_scope(scope):
        nn = layer_in
        for idx in range(num_layers):
            nn = tf.layers.dense(nn, hidden_dim, activation=activation)
        layer_out = tf.layers.dense(nn, output_dim, activation=output_activation)
    return layer_out

In [15]:
def build_g(layer_in, mask_post, mask_prior, scope, k, num_layers, hidden_dim):
    '''
    layer_in, mask = (None, 2)
    '''
    with tf.variable_scope(scope):
        prior = layer_in * mask_prior
        pi = build_mlp(prior, "pi", num_layers, hidden_dim, k, output_activation=tf.nn.softmax)
        mu = build_mlp(prior, "mu", num_layers, hidden_dim, k)
        sigma2 = build_mlp(prior, "sigma2", num_layers, hidden_dim, k)
        norm_dist = tfp.distributions.Normal(0, 1)
        shifted = tf.reduce_sum((layer_in*mask_post - tf.expand_dims(mu, axis=-1))/sigma2, axis=-1)
        print(tf.shape(shifted)) # shold be (None, k)
        layer_out = tf.reduce_sum(norm_dist.get_prob(shifted) * pi, axis=-1, name="layer_out")
    return layer_out

$$\log p_{\theta}(x)=\log p\left(f_{\theta}(x)\right)+\log \left|\operatorname{det} \frac{\partial f_{\theta}(x)}{\partial x}\right|$$
$$f_\theta\left(\mathbf{x}_1\right) := \int_{-\infty}^{\mathbf{x}_1} g_\theta\left(\mathbf{t}\right) d\mathbf{t}$$
$$f_\theta\left(\mathbf{x}_2|\mathbf{x}_1\right) := \int_{-\infty}^{\mathbf{x}_2} g_\theta\left(\mathbf{t}|\mathbf{x}_1\right) d\mathbf{t}$$
$$
\frac{\partial \mathbf{z}}{\partial \mathbf{x}}=\left[ \begin{array}{cc}{g_\theta\left(\mathbf{x}_1\right)} & {0} \\ {g_\theta\left(\mathbf{x}_2|\mathbf{x}_1\right)^T\frac{\partial g_\theta\left(\mathbf{x}_2|\mathbf{x}_1\right)}{\partial \mathbf{x}_1}} & {g_\theta\left(\mathbf{x}_2|\mathbf{x}_1\right)}\end{array}\right]
$$
For uniform base distribution, the first term is 0; the second term is 
$$\left|g_\theta\left(\mathbf{x}_1\right)g_\theta\left(\mathbf{x}_2|\mathbf{x}_1\right)\right|$$
To generate samples, 
$$p_{\theta}(\mathbf{x}_1, \mathbf{x}_2) = g_\theta\left(\mathbf{x}_1\right)g_\theta\left(\mathbf{x}_2|\mathbf{x}_1\right)$$

In [32]:
class MixGaussian():
    def __init__(self, sess, k, num_layers=4, hidden_dim=8, learning_rate=1e-3):
        self.sess = sess
        self.input = tf.placeholder(shape=[None, 2], name="input", dtype=tf.float32)
        self.k = k
        
        mask_post_1 = tf.constant([1.0, .0], dtype=tf.float32, name="mask_post_1")
        mask_prior_1 = tf.constant([.0, .0], dtype=tf.float32, name="mask_prior_1")
        self.g_1 = build_g(self.input, mask_post_1, mask_prior_2, "g_1", k, num_layers, hidden_dim)
        mask_post_2 = tf.constant([.0, 1.0], dtype=tf.float32, name="mask_post_2")
        mask_prior_2 = tf.constant([1.0, .0], dtype=tf.float32, name="mask_prior_2")
        self.g_2 = build_g(mask_post_2, mask_prior_2, "g_2", k, num_layers, hidden_dim)
        
        self.density_model = tf.abs(self.g_1 * self.g_2)
        self.loss = tf.reduce_mean(-tf.log(self.density_model), axis=0)
        print(tf.shape(self.loss)) # should be a scalar
        self.op = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
    
    def step(self, batch, with_update=False):
        if with_update:
            loss, _ = self.sess.run([self.loss, self.op], feed_dict={self.input: batch})
        else:
            loss = self.sess.run(self.loss, feed_dict={self.input: batch})
        return loss
    
    def evaluate(self, samples):
        return self.sess.run(self.density_model, feed_dict={self.input: samples})

In [33]:
def plot_density_model(network, step=0.1):
    x, y = np.arange(-4, 4, step), np.arange(-4, 4, step)
    xv, yv = meshgrid(x, y)
    xv, yv = np.array(xv).flatten(), np.array(yv).flatten()
    z = network.evaluate(list(zip(xv, yv)))
    plt.pcolormesh(x, y, z)

In [34]:
def train(sess, data_trn, data_val, k=6, batch_size=256, num_epochs=5, log_per_epoch=1):
    network = MixGaussian(sess)
    init_op = tf.initializers.global_variables()
    sess.run(init_op)

    loss_trn = []
    loss_val = []

    for epoch in range(num_epochs):
        print('starting epoch', epoch)
        loss_trn_batch = []
        for batch in np.array_split(data_trn, np.ceil(len(data_trn)/batch_size)):
            loss = network.step(batch, with_update=True)
            loss_trn_batch.append(loss)

        if epoch % log_per_epoch == 0:
            loss_trn.append(np.mean(loss_trn_batch))
            loss_val.append(network.step(data_val, with_update=False))
            print(loss_trn[-1], loss_val[-1])
    return loss_trn, loss_val, network

In [37]:
tf.reset_default_graph()
sess = tf.Session()
loss_trn, loss_val, network = train(sess, data_trn, data_val)

NameError: name 'data_trn' is not defined

In [38]:
plot_density_model(network)

NameError: name 'network' is not defined

In [ ]:
sess.close()